# Исследование надёжности заёмщиков
Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [ ]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter
import numpy as np

In [ ]:
#data = pd.read_csv(r'C:\Users\natalie\Desktop\data.csv')
data = pd.read_csv('/datasets/data.csv')
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В выведенных строках таблицы видим первые трудности для дальнейшего анализа: отрицательный стаж, верхний и нижний регистры вперемешку, пропуски. В data.info() видим размер таблицы (21525х12), основные типы данных (float64(2), int64(5), object(5)), наличие пропусков в данных в столбцах days_employed и total_income.


**Описание данных** (из файла с инструкцией по выполнению проекта):
*	children — количество детей в семье
*	days_employed — общий трудовой стаж в днях
*	dob_years — возраст клиента в годах
*	education — уровень образования клиента
*	education_id — идентификатор уровня образования
*	family_status — семейное положение
*	family_status_id — идентификатор семейного положения
*	gender — пол клиента
*	income_type — тип занятости
*	debt — имел ли задолженность по возврату кредитов
*	total_income — ежемесячный доход
*	purpose — цель получения кредита


Чтобы нам было проще ориентироваться в данных, заменим названия столбцов:

In [ ]:
data.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

In [ ]:
data.set_axis(['children', 'employed_days', 'age_years', 'education_type', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt_type',
       'income_total_month', 'purpose'], axis='columns', inplace=True)
data.columns

Index(['children', 'employed_days', 'age_years', 'education_type',
       'education_id', 'family_status', 'family_status_id', 'gender',
       'income_type', 'debt_type', 'income_total_month', 'purpose'],
      dtype='object')

### Дети

In [ ]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

Выделяющиеся значения: -1 ребенок и 20 детей. 
-1 ребенок - это скорее всего ошибка ввода, и там 1 ребенок. 
20 детей - теоретически вероятный вариант. Хочется посмотреть на эту семью (семьи).

In [ ]:
data[data['children']==20].head(10)

,children,employed_days,age_years,education_type,education_id,family_status,family_status_id,gender,income_type,debt_type,income_total_month,purpose
606,20,-880.221113,21,среднее,1,женат / замужем,0,M,компаньон,0,145334.865002,покупка жилья
720,20,-855.595512,44,среднее,1,женат / замужем,0,F,компаньон,0,112998.738649,покупка недвижимости
1074,20,-3310.411598,56,среднее,1,женат / замужем,0,F,сотрудник,1,229518.537004,получение образования
2510,20,-2714.161249,59,высшее,0,вдовец / вдова,2,F,сотрудник,0,264474.835577,операции с коммерческой недвижимостью
2941,20,-2161.591519,0,среднее,1,женат / замужем,0,F,сотрудник,0,199739.941398,на покупку автомобиля
3302,20,NaN,35,среднее,1,Не женат / не замужем,4,F,госслужащий,0,NaN,профильное образование
3396,20,NaN,56,высшее,0,женат / замужем,0,F,компаньон,0,NaN,высшее образование
3671,20,-913.161503,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,101255.492076,на покупку подержанного автомобиля
3697,20,-2907.910616,40,среднее,1,гражданский брак,1,M,сотрудник,0,115380.694664,на покупку подержанного автомобиля
3735,20,-805.044438,26,высшее,0,Не женат / не замужем,4,M,сотрудник,0,137200.646181,ремонт жилью


In [ ]:
print('Семей с 20-ю детьми всего', data[data['children']==20]['children'].count())
print('Семей с 20-ю детьми у родителей в возрасте до 30 лет:', data[(data['children']==20)&(data['age_years']<=30)]['children'].count())

Семей с 20-ю детьми всего 76
Семей с 20-ю детьми у родителей в возрасте до 30 лет: 13


Вероятнее всего, 20 детей в данной таблице - ошибка ввода. Об этом говорит и количество (67 семей), и возраст родителей (21 год, например). Скорее всего 20 в данной ситуации - это 2.

### Трудовой стаж

В общей таблице бросается в глаза большое количество отрицательных значений в этой графе и наличие нереалистичных значений (например, огромный стаж, больше 1000 лет)

Посмотрим, у каких категорий стаж имеет положительные значения:

In [ ]:
data[data['employed_days']>=0]['income_type'].unique()

array(['пенсионер', 'безработный'], dtype=object)

Отрицательные значения у:

In [ ]:
data[data['employed_days']<0]['income_type'].unique()

array(['сотрудник', 'компаньон', 'госслужащий', 'студент',
       'предприниматель', 'в декрете'], dtype=object)

Положительные значения в графе стажа имеют только пенсионеры и безработные. 

Можно предположить, что причина в особенностях внесения данных, например дефисом (он же минус) обозначался незавершенный стаж, а положительный стаж имели те, у кого он на данный момент стоит на месте. 

In [ ]:
print('Средний трудовой стаж  у пенсионеров и безработных: {:.0f}'.format(data[data['employed_days']>=0]['employed_days'].mean()/365, 'лет')) 
print('Максимальное количество лет стажа у пенсионеров и безработных: {:.0f}'.format(data[data['employed_days']>=0]['employed_days'].max()/365, 'лет'))
print('Минимальное количество лет стажа у пенсионеров и безработных: {:.0f}'.format(data[data['employed_days']>=0]['employed_days'].min()/365, 'лет'))

Средний трудовой стаж  у пенсионеров и безработных: 1000
Максимальное количество лет стажа у пенсионеров и безработных: 1101
Минимальное количество лет стажа у пенсионеров и безработных: 901


Выяснили, что средний стаж у пенсионеров и безработных равен 1000 лет. Странно. Нужно разбираться.

Посмотрим, что со стажем у непенсионеров и небезработных, то есть у тех, у кого стаж имеет отрицательные значения.

In [ ]:
print('Средний трудовой стаж  у остальных категорий: {:.2f}'.format(data[data['employed_days']<0]['employed_days'].mean()/365*-1, 'лет')) 
print('Медианный трудовой стаж  у остальных категорий: {:.2f}'.format(data[data['employed_days']<0]['employed_days'].median()/365*-1, 'лет'))
print('Минимальное количество лет стажа у остальных категорий: {:.2f}'.format(data[data['employed_days']<0]['employed_days'].max()/365*-1, 'лет'))
print('Максимальное количество лет стажа у остальных категорий: {:.2f}'.format(data[data['employed_days']<0]['employed_days'].min()/365*-1, 'лет'))

Средний трудовой стаж  у остальных категорий: 6.45
Медианный трудовой стаж  у остальных категорий: 4.47
Минимальное количество лет стажа у остальных категорий: 0.07
Максимальное количество лет стажа у остальных категорий: 50.38


Записи трудового стажа у остальных категорий ('сотрудник', 'компаньон', 'госслужащий', 'студент', 'предприниматель', 'в декрете') выглядят относительно правдоподобно (если убрать "-"). 

Интересно, что есть люди, начавшие свой трудовой путь в 10 лет...

### Возраст заемщика

Посмотрим, нет ли в таблице 1000-летних людей (вдруг это их стаж был).

In [ ]:
data['age_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75])

Возраст - 0 лет - вероятнее всего пропустили при вводе. Заменим потом на средний возраст. Для этого вычислим его, исключив при вычислении нулевой возраст.

In [ ]:
print('Средний возраст потенциального заемщика: {:.0f}'.format(data[data['age_years']>0]['age_years'].mean()), 'года/лет')
print('Максимальный возраст потенциального заемщика: {:.0f}'.format(data[data['age_years']>0]['age_years'].max()), 'года/лет')
print('Минимальный возраст потенциального заемщика: {:.0f}'.format(data[data['age_years']>0]['age_years'].min()), 'года/лет')

Средний возраст потенциального заемщика: 43 года/лет
Максимальный возраст потенциального заемщика: 75 года/лет
Минимальный возраст потенциального заемщика: 19 года/лет


Посмотрим отдельно средний возраст пенсионеров и безработных (чему должен соответствовать средний стаж?)

In [ ]:
print('Средний возраст пенсионеров и безработных: {:.0f}'.format(data[data['employed_days']>0]['age_years'].mean()), 'года/лет')

Средний возраст пенсионеров и безработных: 59 года/лет


Вот самые старшие пенсионеры с неправдоподобным трудовым стажем.

In [ ]:
data[(data['income_type']=='пенсионер')&(data['age_years'])].sort_values(by='age_years', ascending=False).head()

,children,employed_days,age_years,education_type,education_id,family_status,family_status_id,gender,income_type,debt_type,income_total_month,purpose
20610,0,364291.049263,73,среднее,1,вдовец / вдова,2,F,пенсионер,0,64437.765650,покупка жилой недвижимости
6406,0,352239.799751,73,среднее,1,женат / замужем,0,F,пенсионер,0,119190.487878,покупка жилья
19385,0,372290.349044,73,начальное,3,гражданский брак,1,F,пенсионер,0,100166.624237,на проведение свадьбы
1826,0,368375.048770,73,Среднее,1,вдовец / вдова,2,F,пенсионер,0,74284.473064,покупка жилой недвижимости
5804,0,399563.882630,73,среднее,1,женат / замужем,0,F,пенсионер,0,126603.199013,покупка коммерческой недвижимости


### Образование

Посмотрим, какие есть варианты образования.

In [ ]:
data['education_type'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Соответствие типов образования id:

In [ ]:
data[['education_id', 'education_type']].drop_duplicates().reset_index(drop=True)

,education_id,education_type
0,0,высшее
1,1,среднее
2,1,Среднее
3,1,СРЕДНЕЕ
4,0,ВЫСШЕЕ
5,2,неоконченное высшее
6,3,начальное
7,0,Высшее
8,2,НЕОКОНЧЕННОЕ ВЫСШЕЕ
9,2,Неоконченное высшее


Всего типов образования 5. 

Данные будет необходимо привести к одному регистру, но уже сейчас видно соответствие id типу образования.

### Пол потенциального заёмщика

Посмотрим, какие варианты имеются в графе пола заёмщика.

In [ ]:
data['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

Есть непонятный пол XNA. Посмотрим, много ли его в данных и чему он соответствует.

In [ ]:
data[data['gender']=='XNA']['gender'].count()

1

In [ ]:
data[data['gender']=='XNA']

,children,employed_days,age_years,education_type,education_id,family_status,family_status_id,gender,income_type,debt_type,income_total_month,purpose
10701,0,-2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


Вероятно, XNA - это NA (not available - данные отсутствуют). Удалим его на этапе предобработки данных.

### Семейный статус

Посмотрим, какие имеются семейные статусы в данных:

In [ ]:
data[['family_status_id', 'family_status']].drop_duplicates().reset_index(drop=True)

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


Данные по family_status необходимо привести к единому виду (Не женат / не замужем - не женат / не замужем)

### Источник дохода

Варианты источников дохода в данных:

In [ ]:
data['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [ ]:
data.groupby('income_type').count()

,children,employed_days,age_years,education_type,education_id,family_status,family_status_id,gender,debt_type,income_total_month,purpose
income_type,,,,,,,,,,,
безработный,2,2,2,2,2,2,2,2,2,2,2
в декрете,1,1,1,1,1,1,1,1,1,1,1
госслужащий,1459,1312,1459,1459,1459,1459,1459,1459,1459,1312,1459
компаньон,5085,4577,5085,5085,5085,5085,5085,5085,5085,4577,5085
пенсионер,3856,3443,3856,3856,3856,3856,3856,3856,3856,3443,3856
предприниматель,2,1,2,2,2,2,2,2,2,1,2
сотрудник,11119,10014,11119,11119,11119,11119,11119,11119,11119,10014,11119
студент,1,1,1,1,1,1,1,1,1,1,1


Безработных, в декрете и студентов ничтожно мало.

### Размер дохода

In [ ]:
print('Максимальный доход: {:.0f}'.format(data['income_total_month'].max()))
print('Минимальный доход: {:.0f}'.format(data['income_total_month'].min()))
print('Средний доход: {:.0f}'.format(data['income_total_month'].mean()))
print('Медианный доход: {:.0f}'.format(data['income_total_month'].median()))

Максимальный доход: 2265604
Минимальный доход: 20667
Средний доход: 167422
Медианный доход: 145018


### Наличие или отсутствие задолжности

Как обозначается наличие и отсутвие задолжности:

In [ ]:
data['debt_type'].unique()

array([0, 1])

Сколько всего представителей этих двух категорий:

In [ ]:
data.groupby('debt_type').count()

,children,employed_days,age_years,education_type,education_id,family_status,family_status_id,gender,income_type,income_total_month,purpose
debt_type,,,,,,,,,,,
0,19784,17780,19784,19784,19784,19784,19784,19784,19784,17780,19784
1,1741,1571,1741,1741,1741,1741,1741,1741,1741,1571,1741


### Цель кредита

Посмотрим, какие имеются варианты целей в порядке убывания популярности:

In [ ]:
data['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

Целей кредитования очень много. 
Среди них много дублей, которые необходимо устранить.
Так же стоит эти цели категоризировать.

### Вывод

Задачи для предобработки данных:
    
1. Разобраться с пропусками в столбцах days_employed и total_income.
2. В графе, отражающей количество детей у потенциального заёмщика, необходимо исправить ошибки: заменить -1 ребенка и 20 детей на 1 и 2 соответственно.
3. Средний стаж у пенсионеров и безработных равен 1000 лет. Странно. Нужно разбираться.
4. Отрицательный стаж у категорий: 'сотрудник', 'компаньон', 'госслужащий', 'студент', 'предприниматель', 'в декрете' - лишить минуса.
5. Возраст 0 лет заменить на средний возраст.
6. Привести к общему виду (нижний регистр) данные о типах образования и семейных статусах.
7. Пол XNA
8. В графе, отражающей цели кредитования необходимо устранить дубли и категоризировать данные.
9. Категоризировать стаж, возраст, зарплаты.


## 2  Шаг 2. Предобработка данных

### Обработка пропусков

Посмотрим, сколько всего пропусков в данных.

In [ ]:
data.isnull().sum()

children                 0
employed_days         2174
age_years                0
education_type           0
education_id             0
family_status            0
family_status_id         0
gender                   0
income_type              0
debt_type                0
income_total_month    2174
purpose                  0
dtype: int64

Количество пропусков в столбцах employed_days и income_total (стаж и доход) идентично. Скорее всего это пропуски в данных о работе у одних и тех же людей.
Так как речь идет о существенных для выдачи кредита данных, от пропусков мы избавимся (другой возможный вариант - заменить на средние значения).

In [ ]:
#data.dropna(subset=['employed_days', 'income_total_month'], inplace=True)
#data.isnull().sum()

Приведём стаж и возраст к общему знаменателю - году - и переимунуем соответствующий столбец.

In [ ]:
data['employed_days'] = data['employed_days']/365

In [ ]:
data.set_axis(['children', 'employed_years', 'age_years', 'education_type', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt_type',
       'income_total_month', 'purpose'], axis='columns', inplace=True)
data.columns

Index(['children', 'employed_years', 'age_years', 'education_type',
       'education_id', 'family_status', 'family_status_id', 'gender',
       'income_type', 'debt_type', 'income_total_month', 'purpose'],
      dtype='object')

Варианты среднего дохода по типам занятости, на который мы будем заменять пропущенные значения.

In [ ]:
data.groupby('income_type')['income_total_month'].mean()

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        170898.309923
компаньон          202417.461462
пенсионер          137127.465690
предприниматель    499163.144947
сотрудник          161380.260488
студент             98201.625314
Name: income_total_month, dtype: float64

Средний стаж для каждой группы вычислим по корректным данным. На него будем заменять пропуски в каждой группе.

In [ ]:
data[data['employed_years']<0].groupby('income_type').agg({'employed_years' : ['mean']})

,employed_years
,mean
income_type,
в декрете,-9.032219
госслужащий,-9.314786
компаньон,-5.784998
предприниматель,-1.426981
сотрудник,-6.373970
студент,-1.585621


Вычислим средний возраст пенсионеров и безработных, чтобы соотнести со средним стажем заёмщиков других категорий (то есть тех, чьи данные о стаже мы признали правдоподобными)

In [ ]:
data.groupby('income_type')['age_years'].mean()

income_type
безработный        38.000000
в декрете          39.000000
госслужащий        40.636737
компаньон          39.697542
пенсионер          59.063019
предприниматель    42.500000
сотрудник          39.821027
студент            22.000000
Name: age_years, dtype: float64


Средний возраст безработных: 38 лет.
    
Средний возраст пенсионеров: 59 лет.
    
Вычислим средний трудовой стаж, соответствующий этим возрастам, у других категорий.

In [ ]:
print('Cредний трудовой стаж 38-летних непенсионеров и небезработных: {:.0f}'.format(data[(data['age_years']==38)&(data['employed_years']<0)]['employed_years'].mean()*-1), 'лет')
print('Cредний трудовой стаж 59-летних непенсионеров и небезработных: {:.0f}'.format(data[(data['age_years']==59)&(data['employed_years']<0)]['employed_years'].mean()*-1), 'лет')

Cредний трудовой стаж 38-летних непенсионеров и небезработных: 6 лет
Cредний трудовой стаж 59-летних непенсионеров и небезработных: 11 лет



Заменим на них некорректные значения у пенсионеров и безработных.

In [ ]:
data.loc[data['employed_years']>0, 'employed_years'] = '11'

In [ ]:
data[data['income_type']=='безработный'] = data[data['income_type']=='безработный'].replace('11', '6')

In [ ]:
data[data['income_type']=='безработный']

,children,employed_years,age_years,education_type,education_id,family_status,family_status_id,gender,income_type,debt_type,income_total_month,purpose
3133,1,6,31,среднее,1,женат / замужем,0,M,безработный,1,59956.991984,покупка жилья для сдачи
14798,0,6,45,Высшее,0,гражданский брак,1,F,безработный,0,202722.511368,ремонт жилью


In [ ]:
data[data['income_type']=='пенсионер'].head()

,children,employed_years,age_years,education_type,education_id,family_status,family_status_id,gender,income_type,debt_type,income_total_month,purpose
4,0,11,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
18,0,11,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля
24,1,11,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547.235997,операции с коммерческой недвижимостью
25,0,11,67,среднее,1,женат / замужем,0,M,пенсионер,0,55112.757732,покупка недвижимости



Посмотрим в каких именно категориях имеются пропуски.

In [ ]:
print('Пропущенных данных о доходах у безработных:', data[data['income_type']=='безработный']['income_total_month'].isnull().sum())
print('Пропущенных данных о доходах у находящихся в декрете:', data[data['income_type']=='в декрете']['income_total_month'].isnull().sum())
print('Пропущенных данных о доходах у госслужащих:', data[data['income_type']=='госслужащий']['income_total_month'].isnull().sum())
print('Пропущенных данных о доходах у компаньонов:', data[data['income_type']=='компаньон']['income_total_month'].isnull().sum())
print('Пропущенных данных о доходах у пенсионеров:', data[data['income_type']=='пенсионер']['income_total_month'].isnull().sum())
print('Пропущенных данных о доходах у предпринимателей:', data[data['income_type']=='предприниматель']['income_total_month'].isnull().sum())
print('Пропущенных данных о доходах у сотрудников:', data[data['income_type']=='сотрудник']['income_total_month'].isnull().sum())
print('Пропущенных данных о доходах у студентов:', data[data['income_type']=='студент']['income_total_month'].isnull().sum())

Пропущенных данных о доходах у безработных: 0
Пропущенных данных о доходах у находящихся в декрете: 0
Пропущенных данных о доходах у госслужащих: 147
Пропущенных данных о доходах у компаньонов: 508
Пропущенных данных о доходах у пенсионеров: 413
Пропущенных данных о доходах у предпринимателей: 1
Пропущенных данных о доходах у сотрудников: 1105
Пропущенных данных о доходах у студентов: 0


In [ ]:
print('Пропущенных данных о стаже у безработных:', data[data['income_type']=='безработный']['employed_years'].isnull().sum())
print('Пропущенных данных о стаже у находящихся в декрете:', data[data['income_type']=='в декрете']['employed_years'].isnull().sum())
print('Пропущенных данных о стаже у госслужащих:', data[data['income_type']=='госслужащий']['employed_years'].isnull().sum())
print('Пропущенных данных о стаже у компаньонов:', data[data['income_type']=='компаньон']['employed_years'].isnull().sum())
print('Пропущенных данных о стаже у пенсионеров:', data[data['income_type']=='пенсионер']['employed_years'].isnull().sum())
print('Пропущенных данных о стаже у предпринимателей:', data[data['income_type']=='предприниматель']['employed_years'].isnull().sum())
print('Пропущенных данных о стаже у сотрудников:', data[data['income_type']=='сотрудник']['employed_years'].isnull().sum())
print('Пропущенных данных о стаже у студентов:', data[data['income_type']=='студент']['employed_years'].isnull().sum())

Пропущенных данных о стаже у безработных: 0
Пропущенных данных о стаже у находящихся в декрете: 0
Пропущенных данных о стаже у госслужащих: 147
Пропущенных данных о стаже у компаньонов: 508
Пропущенных данных о стаже у пенсионеров: 413
Пропущенных данных о стаже у предпринимателей: 1
Пропущенных данных о стаже у сотрудников: 1105
Пропущенных данных о стаже у студентов: 0


Теперь заменим пропущенные значения в графах дохода и стажа на средние по каждой группе занятости.

In [ ]:
data[data['income_type']=='госслужащий'] = data[data['income_type']=='госслужащий'].fillna({'income_total_month':170898, 'employed_years':9})

In [ ]:
data[data['income_type']=='компаньон'] = data[data['income_type']=='компаньон'].fillna({'income_total_month':202417, 'employed_years':6})

In [ ]:
data[data['income_type']=='пенсионер'] = data[data['income_type']=='пенсионер'].fillna({'income_total_month':137127, 'employed_years':11})

In [ ]:
data[data['income_type']=='предприниматель'] = data[data['income_type']=='предприниматель'].fillna({'income_total_month':499163, 'employed_years':1})

In [ ]:
data[data['income_type']=='сотрудник'] = data[data['income_type']=='сотрудник'].fillna({'income_total_month':161380, 'employed_years':6})

In [ ]:
data.isnull().sum()

children              0
employed_years        0
age_years             0
education_type        0
education_id          0
family_status         0
family_status_id      0
gender                0
income_type           0
debt_type             0
income_total_month    0
purpose               0
dtype: int64

In [ ]:
data.shape

(21525, 12)

Пропусков больше нет. Рамер таблицы прежний.

Заменим в столбце age_years нулевой возраст средним.

In [ ]:
data.loc[data['age_years']==0, 'age_years'] = '43'

In [ ]:
data['age_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51, '43', 59, 29, 60, 55, 58, 71, 22,
       73, 66, 69, 19, 72, 70, 74, 75], dtype=object)

Возраста, равного 0, больше нет в данных.


Удалим строку с гендером XNA

In [ ]:
data[data['gender']=='XNA']

,children,employed_years,age_years,education_type,education_id,family_status,family_status_id,gender,income_type,debt_type,income_total_month,purpose
10701,0,-6.46192,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


In [ ]:
data=data.drop(index=10701).reset_index(drop=True)

In [ ]:
data[data['gender']=='XNA']

,children,employed_years,age_years,education_type,education_id,family_status,family_status_id,gender,income_type,debt_type,income_total_month,purpose


Теперь исправим ситуацию с -1 и 20 детей.
Переведем данные из типа object в тип int, чтобы применить функцию abs() и избавиться от минуса.

In [ ]:
data['children'] = data['children'].astype('int64').abs()

Заменим 20 на 2.

In [ ]:
data.loc[data['children']==20, 'children'] = '2'

In [ ]:
data['children'].unique()

array([1, 0, 3, 2, 4, '2', 5], dtype=object)

Теперь диапазон возрастов у детей заёмщиков 0-5.

### Замена типа данных

Данные в столбцах 'employed_years', 'age_years', 'children' имеют тип object. Изменим его на численный и проверим заодно корректность значений.

In [ ]:
data['employed_years'] = pd.to_numeric(data['employed_years'], errors='raise')

In [ ]:
data['age_years'] = pd.to_numeric(data['age_years'], errors='raise')

In [ ]:
data['children'] = pd.to_numeric(data['children'], errors='raise')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21524 entries, 0 to 21523
Data columns (total 12 columns):
children              21524 non-null int64
employed_years        21524 non-null float64
age_years             21524 non-null int64
education_type        21524 non-null object
education_id          21524 non-null int64
family_status         21524 non-null object
family_status_id      21524 non-null int64
gender                21524 non-null object
income_type           21524 non-null object
debt_type             21524 non-null int64
income_total_month    21524 non-null float64
purpose               21524 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Данные в столбцах 'employed_years', 'age_years', 'children' корректны, тип float64

Приведем данные столбцов 'employed_years', 'age_years', 'income_total_month', 'children' к целочисленному типу данных int и избавим от минуса.

In [ ]:
data[['employed_years', 'age_years', 'income_total_month', 'children']] = data[['employed_years', 'age_years', 'income_total_month', 'children']].astype('int64').abs()

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21524 entries, 0 to 21523
Data columns (total 12 columns):
children              21524 non-null int64
employed_years        21524 non-null int64
age_years             21524 non-null int64
education_type        21524 non-null object
education_id          21524 non-null int64
family_status         21524 non-null object
family_status_id      21524 non-null int64
gender                21524 non-null object
income_type           21524 non-null object
debt_type             21524 non-null int64
income_total_month    21524 non-null int64
purpose               21524 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


In [ ]:
data.head()

,children,employed_years,age_years,education_type,education_id,family_status,family_status_id,gender,income_type,debt_type,income_total_month,purpose
0,1,23,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,11,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,15,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,11,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,11,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


### Обработка дубликатов

Приведём к общему виду (нижний регистр) данные о типах образования, семейных статусах и гендера.

In [ ]:
data['education_type'] = data['education_type'].str.lower()

In [ ]:
data['family_status'] = data['family_status'].str.lower()

In [ ]:
data['gender'] = data['gender'].str.lower()

Посмотрим, получилось ли:

In [ ]:
data['education_type'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [ ]:
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)

In [ ]:
data['gender'].unique()

array(['f', 'm'], dtype=object)

Получилось.

Посмотрим на типы семейных статусов и образования.

In [ ]:
data[['family_status_id', 'family_status']].drop_duplicates().reset_index(drop=True)

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,не женат / не замужем


In [ ]:
data[['education_id', 'education_type']].drop_duplicates().reset_index(drop=True)

,education_id,education_type
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


Посмотрим, есть ли дубликаты в таблице.

In [ ]:
data.duplicated().sum()

71

Теперь они действительно появились. Избавимся от них.

In [ ]:
data = data.drop_duplicates().reset_index(drop=True)

Проверим:

In [ ]:
data.duplicated().sum()

0

**Вывод**

Мы устранили дублирующие друг друга значения с записью в разных регистрах (привели к нижнему). Полных дублей в таблице нет. Есть еще большое количество дубликатов в графе с целью кредита, с ними будем разбираться при помощи лемматизации.

### Лемматизация

Теперь попробуем разобраться с целями кредитования. 

Создадим список целей:

In [ ]:
purpose_list = data['purpose'].unique()
purpose_list

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Разберём список на леммы:

In [ ]:
m = Mystem()
word_list_list = data['purpose'].apply(m.lemmatize)
word_list_list

0                             [покупка,  , жилье, \n]
1                   [приобретение,  , автомобиль, \n]
2                             [покупка,  , жилье, \n]
3                [дополнительный,  , образование, \n]
4                           [сыграть,  , свадьба, \n]
                             ...                     
21448                  [операция,  , с,  , жилье, \n]
21449               [сделка,  , с,  , автомобиль, \n]
21450                              [недвижимость, \n]
21451    [на,  , покупка,  , свой,  , автомобиль, \n]
21452             [на,  , покупка,  , автомобиль, \n]
Name: purpose, Length: 21453, dtype: object

Создадим единый список лемм и подсчитаем их количество:

In [ ]:
word_list = sum(word_list_list, [])
Counter(word_list)

Counter({'покупка': 5896,
         ' ': 33569,
         'жилье': 4460,
         '\n': 21453,
         'приобретение': 461,
         'автомобиль': 4306,
         'дополнительный': 906,
         'образование': 4013,
         'сыграть': 765,
         'свадьба': 2324,
         'операция': 2604,
         'с': 2918,
         'на': 2222,
         'проведение': 768,
         'для': 1289,
         'семья': 638,
         'недвижимость': 6350,
         'коммерческий': 1311,
         'жилой': 1230,
         'строительство': 1878,
         'собственный': 635,
         'подержать': 478,
         'свой': 2230,
         'со': 627,
         'заниматься': 904,
         'сделка': 941,
         'подержанный': 486,
         'получение': 1314,
         'высокий': 1374,
         'профильный': 436,
         'сдача': 651,
         'ремонт': 607})

Список состоит из действий (покупка, строительство, операция, получение и т.д.) и объектов действий. Объекты:
* недвижимость(=жилье),
* автомобиль,
* образование,
* свадьба,
* и добавим как отдельную категорию - ремонт жилья (недвижимости).

Жилую и коммерческую недвижимость было бы здорово разделить, но в таких значениях как, к примеру "операции с недвижимостью", не ясно, о каком типе недвижимости речь. Так что оставим просто недвижимость.
Эти объекты и будем считать целями кредитов. 

### Категоризация данных

#### Категории целей

Создадим новый столбец с категориями целей кредитования.
Для этого напишем функцию, присваивающую значение новой ячейке исходя из наличия в соостветствующей лемматизированной ячейке data['purpose'] искомой леммы. Применим эту функцию к столбцу data['purpose'].

In [ ]:
def purpose_category (row):
    lemmas = m.lemmatize(row)
    if 'ремонт' in lemmas and 'жилье' in lemmas:
        return 'ремонт'
    elif 'ремонт' in lemmas and 'недвижимость' in lemmas:
        return 'ремонт'
    elif 'жилье' in lemmas:
        return 'недвижимость'
    elif 'недвижимость' in lemmas:
        return 'недвижимость'
    elif 'свадьба' in lemmas:
        return 'свадьба'
    elif 'автомобиль' in lemmas:
        return 'автомобиль'
    elif 'образование' in lemmas:
        return 'образование'
    else:
        return 'потеряли цель'
          
    
data['purpose_type'] = data['purpose'].apply(purpose_category)
data['purpose_type'].value_counts()

недвижимость    10203
автомобиль       4306
образование      4013
свадьба          2324
ремонт            607
Name: purpose_type, dtype: int64

Ход с ремонтом обусловлен тем, что слова "ремонт" и "жилье" (или "недвижимость") чаще встречаются вместе. Можно было бы убрать вовсе эту категорию, но тогда люди, планирующие ремонт в квартире, окажутся в одной категории с планирующими покупку квартиры, а они (эти категории) несоотносимы. 

#### Категории количества детей у заёмщика

Выделим три категории семей по количеству детей, учитывая, что количество детей в выборке располагается в диапазоне 0-5. Возьмем классическую схему: 
* бездетные (0), 
* 1-2 ребёнка (1-2), 
* многодетные (больше 3).

Напишем функцию с условием и создадим с ёё помощью новый столбец с категориями количества детей.

In [ ]:
def children_category(row):
    if row == 0:
        return 'бездетные'
    elif row == 1 or row == 2:
        return '1-2 ребёнка'
    elif row >=3:
        return 'многодетные'
    else:
        'количество детей не определёно'
        
data['children_type'] = data['children'].apply(children_category)        
data['children_type'].value_counts()

бездетные      14090
1-2 ребёнка     6983
многодетные      380
Name: children_type, dtype: int64

#### Возрастные категории

In [ ]:
data['age_years'].quantile([0.25,0.5,0.75])

0.25    33.0
0.50    43.0
0.75    53.0
Name: age_years, dtype: float64

<font color='green'>
<p>По квантилям выделим 4 возрастные группы:</p>
<li>1 19-33</li>
<li>2 34-43</li>
<li>3 44-53</li>
<li>4 53-57</li>
</font>

In [ ]:
def age_years_category(row):
    if row <= 33:
        return '1 19-33'
    elif 33 < row <= 43:
        return '2 34-43'
    elif 43 < row <= 53:
        return '3 44-53'
    elif row >53:
        return '4 53-57'
    else:
        'возраст не определён'
        
data['age_years_type'] = data['age_years'].apply(age_years_category)        
data['age_years_type'].value_counts()

2 34-43    5897
1 19-33    5365
4 53-57    5255
3 44-53    4936
Name: age_years_type, dtype: int64

#### Категории размеров дохода

In [ ]:
print(np.percentile(data['income_total_month'], 20))
print(np.percentile(data['income_total_month'], 40))
print(np.percentile(data['income_total_month'], 60))
print(np.percentile(data['income_total_month'], 80))

98536.4
135447.0
162638.2
214622.80000000005



Вычислив перцентили, разделим данные по уровням доходов заёмщиков на 5 категорий:
<ul type="circle">
<li>низкий - не больше 98536</li>
<li>ниже среднего - не больше 135447</li>
<li>средний - не больше 162638</li>
<li>выше среднего - не больше 214622</li>
<li>высокий - больше 214622</li>


In [ ]:
def income_month_category(row):
    if row <= 98536:
        return 'низкий'
    elif 98536 < row <= 135447:
        return 'ниже среднего'
    elif 135447 < row <= 162638:
        return 'средний'
    elif 162638 < row <= 214622:
        return 'выше среднего'
    elif row >214622:
        return 'высокий'
    else:
        'доход не определён'
        
data['income_month_type'] = data['income_total_month'].apply(income_month_category)        
data['income_month_type'].value_counts()

высокий          4291
средний          4291
низкий           4291
выше среднего    4290
ниже среднего    4290
Name: income_month_type, dtype: int64

#### Категории стажа

In [ ]:
#print(np.percentile(data['employed_years'], 25))
#print(np.percentile(data['employed_years'], 50))
#print(np.percentile(data['employed_years'], 75))

In [ ]:
data['employed_years'].quantile([0.25,0.5,0.75])

0.25     2.0
0.50     6.0
0.75    11.0
Name: employed_years, dtype: float64


По квантилям выделим 4 группы трудового стажа:
<li>1 минимальный - не больше 2 лет</li>
<li>2 маленький - не больше 6 лет</li>
<li>3 средний - не больше 11 лет</li>
<li>4 продолжительный - больше 11 лет</li>


Я не стала делать шаг еще мельче, так как смысла дальше уменьшать минимальный стаж нет - наглядности это не прибавит.

In [ ]:
def employed_years_category(row):
    if row <= 2:
        return '1 минимальный'
    elif 2 < row <= 6:
        return '2 маленький'
    elif 6 < row <= 11:
        return '3 средний'
    elif row > 11:
        return '4 продолжительный'
    
    else:
        'стаж не определён'
        
data['employed_years_type'] = data['employed_years'].apply(employed_years_category)        
data['employed_years_type'].value_counts()

3 средний            6938
2 маленький          6471
1 минимальный        5729
4 продолжительный    2315
Name: employed_years_type, dtype: int64

Посмотрим на обновленную таблицу

In [ ]:
data.head(15)

,children,employed_years,age_years,education_type,education_id,family_status,family_status_id,gender,income_type,debt_type,income_total_month,purpose,purpose_type,children_type,age_years_type,income_month_type,employed_years_type
0,1,23,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,покупка жилья,недвижимость,1-2 ребёнка,2 34-43,высокий,4 продолжительный
1,1,11,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля,автомобиль,1-2 ребёнка,2 34-43,ниже среднего,3 средний
2,0,15,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,покупка жилья,недвижимость,бездетные,1 19-33,средний,4 продолжительный
3,3,11,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,дополнительное образование,образование,многодетные,1 19-33,высокий,3 средний
4,0,11,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,сыграть свадьбу,свадьба,бездетные,3 44-53,средний,3 средний
5,0,2,27,высшее,0,гражданский брак,1,m,компаньон,0,255763,покупка жилья,недвижимость,бездетные,1 19-33,высокий,1 минимальный
6,0,7,43,высшее,0,женат / замужем,0,f,компаньон,0,240525,операции с жильем,недвижимость,бездетные,2 34-43,высокий,3 средний
7,0,0,50,среднее,1,женат / замужем,0,m,сотрудник,0,135823,образование,образование,бездетные,3 44-53,средний,1 минимальный
8,2,18,35,высшее,0,гражданский брак,1,f,сотрудник,0,95856,на проведение свадьбы,свадьба,1-2 ребёнка,2 34-43,низкий,4 продолжительный
9,0,5,41,среднее,1,женат / замужем,0,m,сотрудник,0,144425,покупка жилья для семьи,недвижимость,бездетные,2 34-43,средний,2 маленький


**Вывод**

К исходной таблице мы прибавили пять новых столбцов, отражающих категории:
* целей кредитования
* количества детей
* возраста
* размера доходов
* трудового стажа

## Шаг 3. Ответьте на вопросы

- **Есть ли зависимость между наличием детей и возвратом кредита в срок?**

In [ ]:
data.groupby('children_type')['debt_type'].mean()

children_type
1-2 ребёнка    0.092654
бездетные      0.075444
многодетные    0.081579
Name: debt_type, dtype: float64

Больше всего задолжностей у имеющих 1-2 детей.
Меньше всего у многодетных и бездетных.

Посмотрим сводные таблицы.

In [ ]:
data.pivot_table(index=['family_status', 'age_years_type'], columns='children_type', aggfunc={'debt_type':'mean'})

debt_type                      
children_type                        1-2 ребёнка бездетные многодетные
family_status         age_years_type                                  
в разводе             1 19-33           0.089109  0.154930    0.200000
                      2 34-43           0.065574  0.050314    0.000000
                      3 44-53           0.065789  0.076336    0.000000
                      4 53-57           0.076923  0.054795         NaN
вдовец / вдова        1 19-33           0.100000  0.000000    0.000000
                      2 34-43           0.100000  0.043478    0.000000
                      3 44-53           0.107143  0.054878    0.000000
                      4 53-57           0.081081  0.066038         NaN
гражданский брак      1 19-33           0.119205  0.107143    0.080000
                      2 34-43           0.111111  0.092593    0.142857
                      3 44-53           0.115226  0.073381    0.166667
                      4 53-57           0.054054  0.067839         NaN
женат / замужем       1 19-33           0.105101  0.106285    0.052632
                      2 34-43           0.086005  0.074293    0.075472
                      3 44-53           0.057410  0.067966    0.115385
                      4 53-57           0.048980  0.051919    0.000000
не женат / не замужем 1 19-33           0.151515  0.110345    0.666667
                      2 34-43           0.102564  0.098558    0.000000
                      3 44-53           0.067568  0.093434    0.000000
                      4 53-57           0.052632  0.045977         NaN

In [ ]:
data.pivot_table(index='purpose_type', columns='children_type', aggfunc={'debt_type':'mean'})

debt_type                      
children_type 1-2 ребёнка бездетные многодетные
purpose_type                                   
автомобиль       0.110871  0.085413    0.083333
недвижимость     0.083682  0.067906    0.075269
образование      0.105873  0.086677    0.051948
ремонт           0.051020  0.057214    0.222222
свадьба          0.087071  0.075163    0.138889

**Вывод**

Больше всего задолжностей у имеющих 1-2 детей, не состоящих в браке (или в разводе). Семейные заёмщики с 1-2 детьми гораздо чаще возвращают кредиты в срок.
Меньше всего долгов по кредитам у многодетных и бездетных, но среди многодетных имеют большее количество задолжностей разведенные до 33 лет и проживающие в гражданском браке до 57 лет. Из бездетных самые надежные заемщики - вдовцы до 33 лет.


_______________________________________________________________________________________________________________________________



- **Есть ли зависимость между семейным положением и возвратом кредита в срок?**

In [ ]:
data.groupby('family_status')['debt_type'].mean()

family_status
в разводе                0.071130
вдовец / вдова           0.065693
гражданский брак         0.093494
женат / замужем          0.075452
не женат / не замужем    0.097509
Name: debt_type, dtype: float64

Не состоящие в браке заёмщики чаще остальных имеют задолжности по кредитам.
Самые редкие должники - вдовцы.

Обратимся к сводным таблицам.

In [ ]:
data.pivot_table(index=['age_years_type'], columns='family_status', aggfunc={'debt_type':'mean'})

debt_type                                                  \
family_status  в разводе вдовец / вдова гражданский брак женат / замужем   
age_years_type                                                             
1 19-33         0.118644       0.076923         0.111408        0.103780   
2 34-43         0.057637       0.064935         0.102965        0.080658   
3 44-53         0.073529       0.061224         0.084746        0.065728   
4 53-57         0.057402       0.066865         0.066667        0.051582   

                                      
family_status  не женат / не замужем  
age_years_type                        
1 19-33                     0.119295  
2 34-43                     0.099349  
3 44-53                     0.088608  
4 53-57                     0.046512

In [ ]:
data.pivot_table(index='income_month_type', columns='family_status', aggfunc={'debt_type':'mean'})

debt_type                                                  \
family_status     в разводе вдовец / вдова гражданский брак женат / замужем   
income_month_type                                                             
высокий            0.056911       0.053846         0.068627        0.067456   
выше среднего      0.068548       0.087838         0.092883        0.073317   
ниже среднего      0.071749       0.052632         0.106843        0.075487   
низкий             0.078189       0.048327         0.089915        0.080098   
средний            0.080851       0.093596         0.108363        0.081224   

                                         
family_status     не женат / не замужем  
income_month_type                        
высокий                        0.092199  
выше среднего                  0.119508  
ниже среднего                  0.092834  
низкий                         0.082515  
средний                        0.099278

**Вывод**

Больше всего задолжностей встречалось у разведенных и живущих гражданским браком заёмщиков до 33 лет с минимальным стажем, средним доходом.
Меньше всего - у несемейных людей с продолжительным стажем и высоким доходом.

Не состоящие в браке заёмщики чаще остальных имеют задолжности по кредитам, особо выделяются живущие в гражданском браке и имеющие 1-2 детей и многодетные неженатые(незамужние).
Самые редкие должники - вдовцы, особенно многодетные.


_______________________________________________________________________________________________________________________________



- **Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

In [ ]:
data.groupby('income_month_type')['debt_type'].mean()

income_month_type
высокий          0.069914
выше среднего    0.083450
ниже среднего    0.082751
низкий           0.080168
средний          0.089490
Name: debt_type, dtype: float64

Отличились в лучшую сторону только заёмщики с высоким уровнем дохода. У остальных показатели равны.

Посмотрим сводные таблицы.

In [ ]:
data.pivot_table(index='purpose_type', columns='income_month_type', aggfunc={'debt_type':'mean'})

debt_type                                                
income_month_type   высокий выше среднего ниже среднего    низкий   средний
purpose_type                                                               
автомобиль         0.075878      0.104762      0.099211  0.084928  0.103488
недвижимость       0.066440      0.075435      0.070289  0.072851  0.081175
образование        0.085826      0.086902      0.101360  0.095355  0.091130
ремонт             0.050000      0.088235      0.046296  0.036036  0.060606
свадьба            0.052516      0.073009      0.081250  0.087794  0.104701

In [ ]:
data.pivot_table(index='employed_years_type', columns='income_month_type', aggfunc={'debt_type':'mean'})

debt_type                                                
income_month_type     высокий выше среднего ниже среднего    низкий   средний
employed_years_type                                                          
1 минимальный        0.087097      0.110907      0.118168  0.118659  0.121453
2 маленький          0.076551      0.084621      0.089080  0.092428  0.087827
3 средний            0.060114      0.063796      0.055118  0.057971  0.074485
4 продолжительный    0.039724      0.067982      0.050505  0.043578  0.071633

**Вывод**

Больше всего задолжностей у заемщиков со средним и ниже среднего уровнем дохода с минимальным трудовым стажем, взявшие кредиты на автомобили и образование.
Меньше всего задолжностей  у заёмщиков с высоким уровнем дохода, особенно имеющих продолжительный стаж работы.


_______________________________________________________________________________________________________________________________



- **Как разные цели кредита влияют на его возврат в срок?**

In [ ]:
data.groupby('purpose_type')['debt_type'].mean()

purpose_type
автомобиль      0.093590
недвижимость    0.073214
образование     0.092200
ремонт          0.057661
свадьба         0.080034
Name: debt_type, dtype: float64

Чаще всего имеют долги взявшие кредиты на автомобиль и образование.
Лучше всего возвращаются кредиты на ремонт и недвижимость.

In [ ]:
data.pivot_table(index='family_status', columns='purpose_type', aggfunc={'debt_type':'mean'})

debt_type                                             
purpose_type          автомобиль недвижимость образование    ремонт   свадьба
family_status                                                                
в разводе               0.074733     0.065934    0.071429  0.128205       NaN
вдовец / вдова          0.091743     0.050980    0.075377  0.062500       NaN
гражданский брак        0.117512     0.094319    0.148515  0.054545  0.080034
женат / замужем         0.083699     0.070544    0.083237  0.048969       NaN
не женат / не замужем   0.128728     0.082502    0.107452  0.064516       NaN

In [ ]:
data.pivot_table(index='age_years_type', columns='purpose_type', aggfunc={'debt_type':'mean'})

debt_type                                             
purpose_type   автомобиль недвижимость образование    ремонт   свадьба
age_years_type                                                        
1 19-33          0.126437     0.098322    0.131000  0.081871  0.098807
2 34-43          0.107113     0.077089    0.089366  0.061350  0.082409
3 44-53          0.075127     0.067775    0.083333  0.021583  0.079096
4 53-57          0.063771     0.047833    0.064833  0.059701  0.059130

**Вывод**

Цель кредита заметно связана с задолжностями у заёмщиков.
Чаще всего имеют долги неженатые и незамужние или разведенные заёмщики до 33 лет, взявшие кредиты на автомобиль и образование.
Лучше всего возвращаются кредиты на ремонт и недвижимость заёмщики старше 43 лет, состоящие в браке.


_______________________________________________________________________________________________________________________________



**Дополнительные параметры**

Также мы можем посмотреть на параметры стажа и источника дохода.

In [ ]:
data.groupby('employed_years_type')['debt_type'].mean()

employed_years_type
1 минимальный        0.110665
2 маленький          0.085767
3 средний            0.061833
4 продолжительный    0.053132
Name: debt_type, dtype: float64

**Вывод**

Ещё мы можем посмотреть на влияние стажа работы на возвращаемость кредитов. Хуже всего рассчитывают свои финансовые силы заёмщики с минимальным трудовым трудовым стажем.

In [ ]:
data.groupby('income_type')['debt_type'].mean()

income_type
безработный        0.500000
в декрете          1.000000
госслужащий        0.059025
компаньон          0.074059
пенсионер          0.056412
предприниматель    0.000000
сотрудник          0.095724
студент            0.000000
Name: debt_type, dtype: float64

In [ ]:
data[data['income_type']=='в декрете'].shape

(1, 17)

**Вывод**

Хуже всего возвращают кредиты безработные (достаточно очевидно). 
Из остальных больше всего задолжностей у наёмных сотрудников, а меньше всего - у предпринимателей.
Заёмщика в декрете в расчет не берем - он всего один.

## Шаг 4. Общий вывод

Наличие детей у заёмщика негативно влияет на возвращение кредита в срок, особенно если заёмщик не состоит в браке. 
Официальный брак оказывает положительное влияние на возвращаемость кредитов. 
Разведенные и проживающие гражданским браком заёмщики гораздо менее надёжны, независимо от уровня доходов. 
Больше всего задолжностей у заемщиков со средним уровнем дохода.
Так же негативным фактором является минимальный стаж работы и уровень дохода ниже высокого. 
Продолжительный трудовой стаж хорошо сказывается на возвращаемости кредитов. 
Еще одним положительным фактором является собственный бизнес у заёмщика, 
в то время как работа по найму отрицательно сказывается на возвращаемости долга.

**Портрет привлекательного для банка заёмщика:**   
состоит в официальном браке, не имеет детей, старше 33 лет, имеет высокий уровень дохода, продолжительный стаж работы, кредит берёт на недвижимость или ремонт.

**Портрет непривлекательно для банка заёмщика:**  
не состоит в официальном браке (гражданский брак или развод), имеет 1-2 детей, младше 33 лет, имеет средний или низкий уровень дохода, минимальный стаж работы, берёт кредит на покупку автомобиля или образование.

